## **Setup**

https://github.com/chriskhanhtran/bert-for-sentiment-analysis

In [ ]:
import torch

if torch.cuda.is_available():       
    device = torch.device("cuda")
    print(f'There are {torch.cuda.device_count()} GPU(s) available.')
    print('Device name:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
Device name: Tesla P100-PCIE-16GB


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
!pip install transformers

## **Data Processing**

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import regex as re
import matplotlib.pyplot as plt

In [ ]:
import os
os.chdir('/content/drive/My Drive/Colab Notebooks')

In [ ]:
text = pd.read_csv('train_preprocessed.csv',index_col=0)
text.head()

,id,text,label
0,1378212405470240771,current day rolling average exactly year ago n...,pro
1,1378170862151598080,racist nyc demands papers order get vaccine li...,anti
2,1272934425601466371,orange dictator said companies competing creat...,neutral
3,1357599878390677506,coronavirus update new york city chinatown bus...,neutral
4,1357555015095709698,coming soon vaccine passport via covid vaccina...,neutral


In [ ]:
text['label'] = text['label'].replace(['anti','neutral','pro'],[0,1,2]) 

In [ ]:
import random
text = text.sample(frac=1)

In [ ]:
# unbalanced data
train = text[:int(len(text)*0.6)]
validation = text[int(len(text)*0.6):int(len(text)*0.8)]
test = text[int(len(text)*0.8):]

In [ ]:
# undersampled data
train = text[text.label==0][:int(len(text[text.label==0])*0.6)]
train = train.append(text[text.label==1][:int(len(text[text.label==1])*0.2)])
train = train.append(text[text.label==2][:int(len(text[text.label==2])*0.2)])

validation = text[text.label==0][int(len(text[text.label==0])*0.6):int(len(text[text.label==0])*0.8)]
validation = validation.append(text[text.label==1][int(len(text[text.label==1])*0.2):int(len(text[text.label==1])*0.3)])
validation = validation.append(text[text.label==2][int(len(text[text.label==2])*0.2):int(len(text[text.label==2])*0.3)])

test = text[text.label==0][int(len(text[text.label==0])*0.8):]
test = test.append(text[text.label==1][int(len(text[text.label==1])*0.3):int(len(text[text.label==1])*0.4)])
test = test.append(text[text.label==2][int(len(text[text.label==2])*0.3):int(len(text[text.label==2])*0.4)])

In [ ]:
# oversampled data
os_text = text.copy()
for x in range(4):
  os_text = os_text.append(text[text.label==0])

os_text = os_text.sample(frac=1).reset_index(drop=True)

train = os_text[:int(len(os_text)*0.6)]
validation = os_text[int(len(os_text)*0.6):int(len(os_text)*0.8)]
test = os_text[int(len(os_text)*0.8):]

## **BERT Classifier**

In [ ]:
from transformers import BertTokenizer

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create a function to tokenize a set of texts
def preprocessing_for_bert(data):
    """Perform required preprocessing steps for pretrained BERT.
    @param    data (np.array): Array of texts to be processed.
    @return   input_ids (torch.Tensor): Tensor of token ids to be fed to a model.
    @return   attention_masks (torch.Tensor): Tensor of indices specifying which
                  tokens should be attended to by the model.
    """
    # Create empty lists to store outputs
    input_ids = []
    attention_masks = []

    # For every sentence...
    for sent in data:
        # `encode_plus` will:
        #    (1) Tokenize the sentence
        #    (2) Add the `[CLS]` and `[SEP]` token to the start and end
        #    (3) Truncate/Pad sentence to max length
        #    (4) Map tokens to their IDs
        #    (5) Create attention mask
        #    (6) Return a dictionary of outputs
        encoded_sent = tokenizer.encode_plus(
            text=sent,
            add_special_tokens=True,        # Add `[CLS]` and `[SEP]`
            max_length=MAX_LEN,                  # Max length to truncate/pad
            pad_to_max_length=True,         # Pad sentence to max length
            #return_tensors='pt',           # Return PyTorch tensor
            return_attention_mask=True      # Return attention mask
            )
        
        # Add the outputs to the lists
        input_ids.append(encoded_sent.get('input_ids'))
        attention_masks.append(encoded_sent.get('attention_mask'))

    # Convert lists to tensors
    input_ids = torch.tensor(input_ids)
    attention_masks = torch.tensor(attention_masks)

    return input_ids, attention_masks

In [ ]:
# Encode our concatenated data
encoded_tweets = [tokenizer.encode(tweet, add_special_tokens=True) for tweet in text.text]

# Find the maximum length
max_len = max([len(sent) for sent in encoded_tweets])
print('Max length: ', max_len)

Max length:  94


In [ ]:
# Specify `MAX_LEN`
MAX_LEN = 94

# Print sentence 0 and its encoded token ids
token_ids = list(preprocessing_for_bert([text.text[0]])[0].squeeze().numpy())
print('Original: ', text.text[5])
print('Token IDs: ', token_ids)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Original:  new feature alert can use live subway map find vaccination site locations nyc simply tap icon show vaccine site locations contact info check
Token IDs:  [101, 2783, 2154, 5291, 2779, 3599, 2095, 3283, 2047, 3572, 2154, 2353, 2103, 2363, 2034, 17404, 13004, 4030, 4897, 16392, 2154, 2779, 2197, 2258, 2025, 2204, 10638, 2506, 8246, 2149, 2700, 4584, 2145, 5337, 17060, 6001, 6396, 2015, 7792, 28896, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
# Run function `preprocessing_for_bert` on the train set and the validation set
print('Tokenizing data...')
train_inputs, train_masks = preprocessing_for_bert(train.text) # CHANGE TRAIN/TEST DATA
val_inputs, val_masks = preprocessing_for_bert(validation.text) # CHANGE TRAIN/TEST DATA'''

Tokenizing data...


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

# Convert other data types to torch.Tensor
train_labels = torch.tensor(list(train.label)) # CHANGE TRAIN/TEST DATA
val_labels = torch.tensor(list(validation.label)) # CHANGE TRAIN/TEST DATA

# For fine-tuning BERT, the authors recommend a batch size of 16 or 32.
batch_size = 16

# Create the DataLoader for our training set
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set
val_data = TensorDataset(val_inputs, val_masks, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

In [ ]:
%%time
import torch
import torch.nn as nn
from transformers import BertModel

# Create the BertClassfier class
class BertClassifier(nn.Module):
    """Bert Model for Classification Tasks.
    """
    def __init__(self, freeze_bert=False):
        """
        @param    bert: a BertModel object
        @param    classifier: a torch.nn.Module classifier
        @param    freeze_bert (bool): Set `False` to fine-tune the BERT model
        """
        super(BertClassifier, self).__init__()
        # Specify hidden size of BERT, hidden size of our classifier, and number of labels
        D_in, H, D_out = 768, 50, 3

        # Instantiate BERT model
        self.bert = BertModel.from_pretrained('bert-base-uncased')

        # Instantiate an one-layer feed-forward classifier
        self.classifier = nn.Sequential(
            nn.Linear(D_in, H),
            nn.ReLU(),
            #nn.Dropout(0.5),
            nn.Linear(H, D_out)
        )

        # Freeze the BERT model
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
        
    def forward(self, input_ids, attention_mask):
        """
        Feed input to BERT and the classifier to compute logits.
        @param    input_ids (torch.Tensor): an input tensor with shape (batch_size,
                      max_length)
        @param    attention_mask (torch.Tensor): a tensor that hold attention mask
                      information with shape (batch_size, max_length)
        @return   logits (torch.Tensor): an output tensor with shape (batch_size,
                      num_labels)
        """
        # Feed input to BERT
        outputs = self.bert(input_ids=input_ids,
                            attention_mask=attention_mask)
        
        # Extract the last hidden state of the token `[CLS]` for classification task
        last_hidden_state_cls = outputs[0][:, 0, :]

        # Feed input to classifier to compute logits
        logits = self.classifier(last_hidden_state_cls)

        return logits

CPU times: user 40 µs, sys: 0 ns, total: 40 µs
Wall time: 43.9 µs


In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup

def initialize_model(epochs=4):
    """Initialize the Bert Classifier, the optimizer and the learning rate scheduler.
    """
    # Instantiate Bert Classifier
    bert_classifier = BertClassifier(freeze_bert=False)

    # Tell PyTorch to run the model on GPU
    bert_classifier.to(device)

    # Create the optimizer
    optimizer = AdamW(bert_classifier.parameters(),
                      lr=5e-5,    # Default learning rate
                      eps=1e-8    # Default epsilon value
                      )

    # Total number of training steps
    total_steps = len(train_dataloader) * epochs

    # Set up the learning rate scheduler
    scheduler = get_linear_schedule_with_warmup(optimizer,
                                                num_warmup_steps=0, # Default value
                                                num_training_steps=total_steps)
    return bert_classifier, optimizer, scheduler

In [ ]:
import random
import time

# Specify loss function
loss_fn = nn.CrossEntropyLoss()

def set_seed(seed_value=42):
    """Set seed for reproducibility.
    """
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)

def train(model, train_dataloader, val_dataloader=None, epochs=4, evaluation=False):
    """Train the BertClassifier model.
    """
    # Start training loop
    print("Start training...\n")
    for epoch_i in range(epochs):
        # =======================================
        #               Training
        # =======================================
        # Print the header of the result table
        print(f"{'Epoch':^7} | {'Batch':^7} | {'Train Loss':^12} | {'Val Loss':^10} | {'Val Acc':^9} | {'Elapsed':^9}")
        print("-"*70)

        # Measure the elapsed time of each epoch
        t0_epoch, t0_batch = time.time(), time.time()

        # Reset tracking variables at the beginning of each epoch
        total_loss, batch_loss, batch_counts = 0, 0, 0

        # Put the model into the training mode
        model.train()

        # For each batch of training data...
        for step, batch in enumerate(train_dataloader):
            batch_counts +=1
            # Load batch to GPU
            b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

            # Zero out any previously calculated gradients
            model.zero_grad()

            # Perform a forward pass. This will return logits.
            logits = model(b_input_ids, b_attn_mask)

            # Compute loss and accumulate the loss values
            loss = loss_fn(logits, b_labels)
            batch_loss += loss.item()
            total_loss += loss.item()

            # Perform a backward pass to calculate gradients
            loss.backward()

            # Clip the norm of the gradients to 1.0 to prevent "exploding gradients"
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

            # Update parameters and the learning rate
            optimizer.step()
            scheduler.step()

            # Print the loss values and time elapsed for every 20 batches
            if (step % 20 == 0 and step != 0) or (step == len(train_dataloader) - 1):
                # Calculate time elapsed for 20 batches
                time_elapsed = time.time() - t0_batch

                # Print training results
                print(f"{epoch_i + 1:^7} | {step:^7} | {batch_loss / batch_counts:^12.6f} | {'-':^10} | {'-':^9} | {time_elapsed:^9.2f}")

                # Reset batch tracking variables
                batch_loss, batch_counts = 0, 0
                t0_batch = time.time()

        # Calculate the average loss over the entire training data
        avg_train_loss = total_loss / len(train_dataloader)

        print("-"*70)
        # =======================================
        #               Evaluation
        # =======================================
        if evaluation == True:
            # After the completion of each training epoch, measure the model's performance
            # on our validation set.
            val_loss, val_accuracy = evaluate(model, val_dataloader)

            # Print performance over the entire training data
            time_elapsed = time.time() - t0_epoch
            
            print(f"{epoch_i + 1:^7} | {'-':^7} | {avg_train_loss:^12.6f} | {val_loss:^10.6f} | {val_accuracy:^9.2f} | {time_elapsed:^9.2f}")
            print("-"*70)
        print("\n")
    
    print("Training complete!")


def evaluate(model, val_dataloader):
    """After the completion of each training epoch, measure the model's performance
    on our validation set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    # Tracking variables
    val_accuracy = []
    val_loss = []

    # For each batch in our validation set...
    for batch in val_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask, b_labels = tuple(t.to(device) for t in batch)

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)

        # Compute loss
        loss = loss_fn(logits, b_labels)
        val_loss.append(loss.item())

        # Get the predictions
        preds = torch.argmax(logits, dim=1).flatten()

        # Calculate the accuracy rate
        accuracy = (preds == b_labels).cpu().numpy().mean() * 100
        val_accuracy.append(accuracy)

    # Compute the average accuracy and loss over the validation set.
    val_loss = np.mean(val_loss)
    val_accuracy = np.mean(val_accuracy)

    return val_loss, val_accuracy

In [ ]:
''' MODEL FOR UNBALANCED DATA '''
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=4)
train(bert_classifier, train_dataloader, val_dataloader, epochs=4, evaluation=True)


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   0.897026   |     -      |     -     |   10.29  
   1    |   29    |   0.800312   |     -      |     -     |   4.12   
----------------------------------------------------------------------
   1    |    -    |   0.868012   |  0.791752  |   66.06   |   16.10  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   2    |   20    |   0.700706   |     -      |     -     |   10.35  
   2    |   29    |   0.692547   |     -      |     -     |   4.23   
----------------------------------------------------------------------
   2    |    -    |   0.698258   |  0.792095  |   65.05   |   16.30  
---------------------------------------------------------------

In [ ]:
''' MODEL FOR OVERSAMPLED DATA '''
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=4)
train(bert_classifier, train_dataloader, val_dataloader, epochs=4, evaluation=True)

Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   20    |   1.046694   |     -      |     -     |   3.55   
   1    |   40    |   0.886322   |     -      |     -     |   3.38   
   1    |   60    |   0.745695   |     -      |     -     |   3.38   
   1    |   73    |   0.727356   |     -      |     -     |   2.10   
----------------------------------------------------------------------
   1    |    -    |   0.865899   |  0.641730  |   75.00   |   13.63  
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   2    |   20    |   0.527992   |     -      |     -     |   3.55   
   2    |   40    |   0.525434   |     -      |     -     |   3.38   
   2    |   60    |   0.467659   |     -      |     -     |   3.3

In [ ]:
''' MODEL FOR UNDERSAMPLED DATA '''
set_seed(42)    # Set seed for reproducibility
bert_classifier, optimizer, scheduler = initialize_model(epochs=4)
train(bert_classifier, train_dataloader, val_dataloader, epochs=4, evaluation=True)


Start training...

 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   1    |   10    |   1.025481   |     -      |     -     |   5.82   
----------------------------------------------------------------------
   1    |    -    |   1.025481   |  0.925957  |   52.60   |   6.80   
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
----------------------------------------------------------------------
   2    |   10    |   0.880080   |     -      |     -     |   5.67   
----------------------------------------------------------------------
   2    |    -    |   0.880080   |  0.853893  |   56.42   |   6.65   
----------------------------------------------------------------------


 Epoch  |  Batch  |  Train Loss  |  Val Loss  |  Val Acc  |  Elapsed 
------------------------------------------------------------

In [ ]:
import torch.nn.functional as F
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn import metrics

def bert_predict(model, test_dataloader):
    """Perform a forward pass on the trained BERT model to predict probabilities
    on the test set.
    """
    # Put the model into the evaluation mode. The dropout layers are disabled during
    # the test time.
    model.eval()

    all_logits = []

    # For each batch in our test set...
    for batch in test_dataloader:
        # Load batch to GPU
        b_input_ids, b_attn_mask = tuple(t.to(device) for t in batch)[:2]

        # Compute logits
        with torch.no_grad():
            logits = model(b_input_ids, b_attn_mask)
        all_logits.append(logits)
    
    # Concatenate logits from each batch
    all_logits = torch.cat(all_logits, dim=0)

    # Apply softmax to calculate probabilities
    probs = F.softmax(all_logits, dim=1).cpu().numpy()
    classes = np.argmax(probs, axis=1)

    return probs, classes

In [ ]:
# UNBALANCED CONFUSION MATRIX
test_inputs, test_masks = preprocessing_for_bert(test.text)
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

probs, classes = bert_predict(bert_classifier, test_dataloader)
accuracy = accuracy_score(test.label,classes)
print(accuracy,"\n")

cm = pd.DataFrame(confusion_matrix(test.label,classes),columns=['anti','neutral','pro'],index=['pred_anti','pred_neutral','pred_pro'])
print(cm)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0.6687898089171974 

              anti  neutral  pro
pred_anti        0       10    6
pred_neutral     0      146   43
pred_pro         0       45   64


In [ ]:
# UNDERSAMPLED CONFUSION MATRIX
test_inputs, test_masks = preprocessing_for_bert(test.text)
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

probs, classes = bert_predict(bert_classifier, test_dataloader)
accuracy = accuracy_score(test.label,classes)
print(accuracy,"\n")

cm = pd.DataFrame(confusion_matrix(test.label,classes),columns=['anti','neutral','pro'],index=['pred_anti','pred_neutral','pred_pro'])
print(cm)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0.6167664670658682 

              anti  neutral  pro
pred_anti        8        9    3
pred_neutral     7       75    7
pred_pro         5       33   20


In [ ]:
# OVERSAMPLED CONFUSION MATRIX
test_inputs, test_masks = preprocessing_for_bert(test.text)
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

probs, classes = bert_predict(bert_classifier, test_dataloader)
accuracy = accuracy_score(test.label,classes)
print(accuracy,"\n")

cm = pd.DataFrame(confusion_matrix(test.label,classes),columns=['anti','neutral','pro'],index=['pred_anti','pred_neutral','pred_pro'])
print(cm,'\n')

print(metrics.classification_report(test.label,classes,digits=4),'\n')

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


0.7877237851662404 

              anti  neutral  pro
pred_anti       92        6    0
pred_neutral     1      145   37
pred_pro         2       37   71 

              precision    recall  f1-score   support

           0     0.9684    0.9388    0.9534        98
           1     0.7713    0.7923    0.7817       183
           2     0.6574    0.6455    0.6514       110

    accuracy                         0.7877       391
   macro avg     0.7990    0.7922    0.7955       391
weighted avg     0.7887    0.7877    0.7880       391
 



## **Classification**

In [ ]:
all_tweets = pd.read_csv('all_tweets_labelled.csv',index_col=0)
all_tweets.head()

,id,text,ref_author_id,label
0,1319419539420053506,gotten flu shot yet stopped plaza del sol fami...,26583978.0,neutral
1,1319412029317414913,lol company ended office lease nyc plans signi...,NaN,neutral
2,1319404343603482625,gotten flu shot yet stopped plaza del sol fami...,26583978.0,neutral
3,1319402845741875200,gotten flu shot yet stopped plaza del sol fami...,26583978.0,neutral
4,1319400472273444865,together love effort sf nyc unless everyone le...,95292805.0,anti


In [ ]:
# Create the DataLoader for our test set
test_inputs, test_masks = preprocessing_for_bert(all_tweets.text)
test_dataset = TensorDataset(test_inputs, test_masks)
test_sampler = SequentialSampler(test_dataset)
test_dataloader = DataLoader(test_dataset, sampler=test_sampler, batch_size=32)

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [ ]:
probs, classes = bert_predict(bert_classifier, test_dataloader)

In [ ]:
all_tweets_labelled = all_tweets.copy().rename(columns={'label':'nltk_label'})
all_tweets_labelled['bert_label'] = classes

In [ ]:
all_tweets_labelled['bert_label'] = all_tweets_labelled['bert_label'].replace([0,1,2],['anti','neutral','pro']) 

In [ ]:
accuracy = accuracy_score(all_tweets_labelled.nltk_label,all_tweets_labelled.bert_label)
print(accuracy,"\n")

cm = pd.DataFrame(confusion_matrix(all_tweets_labelled.nltk_label,all_tweets_labelled.bert_label),columns=['n_anti','n_neutral','n_pro'],index=['b_anti','b_neutral','b_pro'])
print(cm)

0.670828272092317 

           n_anti  n_neutral  n_pro
b_anti       6697       6158   3982
b_neutral    5156     113745  40603
b_pro        1579      29239  56281


In [ ]:
all_tweets_labelled.head()

,id,text,ref_author_id,nltk_label,bert_label
0,1319419539420053506,gotten flu shot yet stopped plaza del sol fami...,26583978.0,neutral,pro
1,1319412029317414913,lol company ended office lease nyc plans signi...,NaN,neutral,neutral
2,1319404343603482625,gotten flu shot yet stopped plaza del sol fami...,26583978.0,neutral,pro
3,1319402845741875200,gotten flu shot yet stopped plaza del sol fami...,26583978.0,neutral,pro
4,1319400472273444865,together love effort sf nyc unless everyone le...,95292805.0,anti,pro


In [ ]:
for label in ['anti','pro','neutral']:
  print(label,"count:",len(all_tweets_labelled[all_tweets_labelled.bert_label==label]))

anti count: 13432
pro count: 100866
neutral count: 149142


In [ ]:
all_tweets_labelled.to_csv('bert_tweets_labelled.csv')